## Script pour récupérer les évenements

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

events = []

for nb in range(6):
    url = 'http://parismoyenage.fr/chrono'+str(nb+1)+'.html'
    print(url)
    r = requests.get(url)
    
    s = str(r.text)
        
    soup_html = BeautifulSoup(s)
    
    list_table = []
    list_table = soup_html.find_all("table")
    # C'est la 3e table de la page qui contient les évenements
    list_table[3]
    
    soup_table = BeautifulSoup(str(list_table[3]))
    
    list_tr = soup_table.find_all("tr")
    
    for tr in list_tr:
        event = {}
        soup2 = BeautifulSoup(str(tr))
        list_td = soup2.find_all("td")
        i = 0
        for td in list_td:
            if(i == 0):
                event["date"] = td.text
            if(i == 1):
                event["evenement"] = td.text
            i = i + 1
        events.append(event)

In [ ]:
len(events)

In [ ]:
pd.options.display.max_rows = 5000
pd.options.display.max_colwidth = 5000

In [ ]:
df = pd.DataFrame(events)

In [ ]:
df

In [ ]:
manips = []
for index, row in df.iterrows():
    manip = {}
    manip["date"] = row["date"]
    manip["evenement"] = row["evenement"]
    
    x = row["date"].split(" ")
    y = x[len(x)-1].split("-")
    manip["startDate"] = y[0]
    if(len(y) > 1):
        manip["endDate"] = y[1]
    else:
        manip["endDate"] = y[0]
    manips.append(manip)

In [ ]:
df2 = pd.DataFrame(manips)

In [ ]:
df2

In [ ]:
df2.dtypes

In [ ]:
toto = pd.to_numeric(df2["startDate"], errors="coerce")
tutu = pd.to_numeric(df2["endDate"], errors="coerce")
df2["startDate"] = toto
df2["endDate"] = tutu

In [ ]:
df2[df2["startDate"] > 1300]

In [ ]:
df2.to_csv("../csv/parismoyenage/extract-evenements-paris.csv", index=False)

## Script pour récupérer les personnages issus des évenements

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import HTTPError
import time
import spacy
from spacy import displacy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
df = pd.read_csv("../csv/parismoyenage/extract-evenements-paris.csv")

In [ ]:
df

In [ ]:
list_evts = ""
for index, row in df.iterrows():
    list_evts = list_evts + " " + row["evenement"]

In [ ]:
list_evts

In [ ]:
nlp_evts = nlp(list_evts)

In [ ]:
displacy.render(nlp_evts, style="ent", jupyter=True)

In [ ]:
personnalites = []
for word in nlp_evts.ents:
    if(word.label_ == "PER"):
        pers = {}
        pers = word.text.lower()
        personnalites.append(pers)

In [ ]:
s = pd.Series(personnalites, name='personnalites')
s = s.drop_duplicates()
s = s.reset_index()
s = s["personnalites"]
len(s)

In [ ]:
persos = []
for i in range(len(s)):
    print(s[i].replace("'"," "))
    try:
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""
        SELECT ?item ?itemLabel ?dateOfBirth ?dateOfDeath ?placeOfBirthLabel ?placeOfDeathLabel (GROUP_CONCAT(DISTINCT ?positionLabel;separator=", ") AS ?positions) 
        WHERE {
          SERVICE wikibase:mwapi {
              bd:serviceParam wikibase:api "EntitySearch" .
              bd:serviceParam wikibase:endpoint "www.wikidata.org" .
              bd:serviceParam mwapi:search '"""+s[i].replace("'"," ")+"""'.
              bd:serviceParam mwapi:language "fr" .
              ?item wikibase:apiOutputItem mwapi:item .
              ?num wikibase:apiOrdinal true .
          }
          ?item wdt:P31 wd:Q5.
          {?item wdt:P569 ?dateOfBirth.} UNION {MINUS {?item wdt:P569 ?dateOfBirth.}}
          {?item wdt:P570 ?dateOfDeath.} UNION {MINUS {?item wdt:P570 ?dateOfDeath.}}
          {?item wdt:P19 ?placeOfBirth.} UNION {MINUS {?item wdt:P19 ?placeOfBirth.}}
          {?item wdt:P20 ?placeOfDeath.} UNION {MINUS {?item wdt:P20 ?placeOfDeath.}}
          {?item wdt:P39 ?position.} UNION {MINUS {?item wdt:P39 ?position.}}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". 
                                  ?item rdfs:label ?itemLabel .
                                 ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                                 ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                                 ?position rdfs:label ?positionLabel .}

        } GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel
        LIMIT 1
        """)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        for r in results['results']['bindings']:
            perso = {}
            if("itemLabel" in r):
                perso['nom'] = r['itemLabel']['value']
            if("dateOfBirth" in r):
                perso['dateOfBirth'] = r['dateOfBirth']['value']
            if("placeOfBirthLabel" in r):
                perso['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
            if("dateOfDeath" in r):
                perso['dateOfDeath'] = r['dateOfDeath']['value']
            if("placeOfDeathLabel" in r):
                perso['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
            if("positions" in r):
                perso["positions"] = r["positions"]["value"]
            persos.append(perso)
    except HTTPError as err:
        print("except - try again")
        i = i-1

In [ ]:
df_persos = pd.DataFrame(persos)
df_persos

In [ ]:
df_persos.to_csv("../csv/parismoyenage/extract-persos-evenements-paris.csv", index=False)

## Script pour récupérer les personnages stockés dans parismoyenage

In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import HTTPError
import time
import spacy
from spacy import displacy
nlp = spacy.load("fr_core_news_sm")

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'http://parismoyenage.fr/parisiens/metier.php'
print(url)
r = requests.get(url)

soup_html = BeautifulSoup(r.text)

list_table = []
list_table = soup_html.find_all("table")
# C'est la 3e table de la page qui contient les évenements

http://parismoyenage.fr/parisiens/metier.php


In [3]:
list_table[0]

<table width="100%">
<tr>
<td bgcolor="#00469B" colspan="100%" height="60" valign="top"> <style type="text/css">
<!--
.style1 {
	font-size: x-large;
	font-family: Arial, Helvetica, sans-serif;
}
-->
</style>
<table bgcolor="#00469B" height="100" width="100%">
<tr>
<td align="left" width="10%">
<img src="blason2.jpg" width="100"/> </td>
<td bgcolor="#00469B">
<div align="center" style="font-size:46px; color:#FFFFFF; "> PARIS AU MOYEN ÂGE<br/> 
			  Les Parisiens  (XIII<sup>e</sup> - XV<sup>e </sup>siècles)</div>
</td>
<td align="right" width="10%">
<img src="blason2.jpg" width="100"/>
</td>
</tr>
</table>
</td>
</tr>
<td valign="top" width="20%">
<p class="style3" style="margin:0">Sélectionnez un métier ou un office dans   l'arborescence:</p>
<div id="menu">
<ul class="niveau1">
<li class="sousmenu">Artisanat, commerce, exploitation
<ul class="niveau2">
<li class="sousmenu">NÃ©goce des mÃ©taux et objets prÃ©cieux
<ul class="niveau3">
<li><a href="metier.php?metier=22"><u>Changeurs</u></

In [5]:
soup_a = BeautifulSoup(str(list_table[0]))
    
list_a = soup_a.find_all("a")
    
for a in list_a:
    print(a['href'])

metier.php?metier=22
metier.php?metier=26
metier.php?metier=27
metier.php?metier=25
metier.php?metier=30
metier.php?metier=31
metier.php?metier=32
metier.php?metier=33
metier.php?metier=512
metier.php?metier=36
metier.php?metier=37
metier.php?metier=38
metier.php?metier=35
metier.php?metier=39
metier.php?metier=40
metier.php?metier=41
metier.php?metier=42
metier.php?metier=501
metier.php?metier=43
metier.php?metier=44
metier.php?metier=57
metier.php?metier=149
metier.php?metier=13
metier.php?metier=58
metier.php?metier=60
metier.php?metier=61
metier.php?metier=62
metier.php?metier=59
metier.php?metier=85
metier.php?metier=86
metier.php?metier=87
metier.php?metier=281
metier.php?metier=88
metier.php?metier=89
metier.php?metier=192
metier.php?metier=523
metier.php?metier=97
metier.php?metier=94
metier.php?metier=95
metier.php?metier=96
metier.php?metier=360
metier.php?metier=524
metier.php?metier=98
metier.php?metier=409
metier.php?metier=99
metier.php?metier=100
metier.php?metier=101
me

In [6]:
a

<a href="javascript:OuvrirPopup('lexique.html', 'lexique', 'resizable=yes, location=no, width=500, height=650, menubar=no, status=no, scrollbars=yes, menubar=no')"><span class="lien">Lexique des métiers</span></a>

In [7]:
url = 'http://parismoyenage.fr/parisiens/metier.php?metier=22'
print(url)
r = requests.get(url)

soup_html = BeautifulSoup(r.text)

list_table = []
list_table = soup_html.find_all("table")
# C'est la 3e table de la page qui contient les évenements

http://parismoyenage.fr/parisiens/metier.php?metier=22


In [11]:
list_table[3]

<table align="center" border="1" style="border-collapse:collapse" width="100%"><tr><td>Prénom</td><td><b><a href="metier.php?metier=22&amp;tri=nom"><span style="color:black">Nom</span></a></b></td><td align="center" width="10%"><a href="metier.php?metier=22&amp;tri=de"><span style="color:black">de</span></a></td><td align="center" width="10%">à</td><td>Fonction</td><td width="15%">Plus d'infos</td></tr><tr><td>Nicolas</td><td><b>Arnoul</b></td><td align="center">1468</td><td align="center"></td><td>changeur</td><td><a href="fiche.php?nom=Arnoul&amp;prenom=Nicolas" target="_blank">Fiche</a></td></tr><tr><td>Jean</td><td><b>Arrode [4]</b></td><td align="center">1353</td><td align="center"></td><td>changeur</td><td><a href="fiche.php?nom=Arrode&amp;prenom=Jean&amp;id=04" target="_blank">Fiche</a></td></tr><tr><td>Nicolas</td><td><b>Arrode [7]</b></td><td align="center">1349</td><td align="center"></td><td>changeur</td><td><a href="fiche.php?nom=Arrode&amp;prenom=Nicolas&amp;id=07" target=

In [26]:

persos = []

for nb in range(len(list_a)-3):
    url = 'http://parismoyenage.fr/parisiens/'+list_a[nb]['href']
    #url = 'http://parismoyenage.fr/parisiens/'+'metier.php?metier=7'
    print(url)
    r = requests.get(url)
    
    s = str(r.text)
        
    soup_html = BeautifulSoup(s)
    
    list_table = []
    list_table = soup_html.find_all("table")
    # C'est la 3e table de la page qui contient les évenements
    if(len(list_table) >= 4):
        soup_table = BeautifulSoup(str(list_table[3]))

        list_tr = soup_table.find_all("tr")

        for tr in list_tr:
            perso = {}
            soup2 = BeautifulSoup(str(tr))
            list_td = soup2.find_all("td")
            i = 0
            for td in list_td:
                if(i == 0):
                    perso["prenom"] = td.text
                if(i == 1):
                    perso["familyName"] = td.text
                    perso["nom"] = perso["prenom"] + " " + td.text
                if(i == 2):
                    perso["startDate"] = td.text
                if(i == 3):
                    perso["endDate"] = td.text
                if(i == 4):
                    perso["positions"] = td.text
                if(i == 5):
                    perso["urlFiche"] = td.text
                i = i + 1
            persos.append(perso)
    else:
        print("pb recup : " + url)

http://parismoyenage.fr/parisiens/metier.php?metier=22
http://parismoyenage.fr/parisiens/metier.php?metier=26
http://parismoyenage.fr/parisiens/metier.php?metier=27
http://parismoyenage.fr/parisiens/metier.php?metier=25
http://parismoyenage.fr/parisiens/metier.php?metier=30
http://parismoyenage.fr/parisiens/metier.php?metier=31
http://parismoyenage.fr/parisiens/metier.php?metier=32
http://parismoyenage.fr/parisiens/metier.php?metier=33
http://parismoyenage.fr/parisiens/metier.php?metier=512
http://parismoyenage.fr/parisiens/metier.php?metier=36
http://parismoyenage.fr/parisiens/metier.php?metier=37
http://parismoyenage.fr/parisiens/metier.php?metier=38
http://parismoyenage.fr/parisiens/metier.php?metier=35
http://parismoyenage.fr/parisiens/metier.php?metier=39
http://parismoyenage.fr/parisiens/metier.php?metier=40
http://parismoyenage.fr/parisiens/metier.php?metier=41
http://parismoyenage.fr/parisiens/metier.php?metier=42
http://parismoyenage.fr/parisiens/metier.php?metier=501
http://p

http://parismoyenage.fr/parisiens/metier.php?metier=240
http://parismoyenage.fr/parisiens/metier.php?metier=241
http://parismoyenage.fr/parisiens/metier.php?metier=242
http://parismoyenage.fr/parisiens/metier.php?metier=173
http://parismoyenage.fr/parisiens/metier.php?metier=394
http://parismoyenage.fr/parisiens/metier.php?metier=84
http://parismoyenage.fr/parisiens/metier.php?metier=180
http://parismoyenage.fr/parisiens/metier.php?metier=243
http://parismoyenage.fr/parisiens/metier.php?metier=244
http://parismoyenage.fr/parisiens/metier.php?metier=245
http://parismoyenage.fr/parisiens/metier.php?metier=228
http://parismoyenage.fr/parisiens/metier.php?metier=246
http://parismoyenage.fr/parisiens/metier.php?metier=19
http://parismoyenage.fr/parisiens/metier.php?metier=350
http://parismoyenage.fr/parisiens/metier.php?metier=130
http://parismoyenage.fr/parisiens/metier.php?metier=131
http://parismoyenage.fr/parisiens/metier.php?metier=132
http://parismoyenage.fr/parisiens/metier.php?metie

http://parismoyenage.fr/parisiens/metier.php?metier=223
http://parismoyenage.fr/parisiens/metier.php?metier=225
http://parismoyenage.fr/parisiens/metier.php?metier=226
http://parismoyenage.fr/parisiens/metier.php?metier=525
http://parismoyenage.fr/parisiens/metier.php?metier=286
http://parismoyenage.fr/parisiens/metier.php?metier=389
http://parismoyenage.fr/parisiens/metier.php?metier=287
http://parismoyenage.fr/parisiens/metier.php?metier=124
http://parismoyenage.fr/parisiens/metier.php?metier=290
http://parismoyenage.fr/parisiens/metier.php?metier=292
http://parismoyenage.fr/parisiens/metier.php?metier=388
http://parismoyenage.fr/parisiens/metier.php?metier=282
http://parismoyenage.fr/parisiens/metier.php?metier=293
http://parismoyenage.fr/parisiens/metier.php?metier=295
http://parismoyenage.fr/parisiens/metier.php?metier=296
http://parismoyenage.fr/parisiens/metier.php?metier=297
http://parismoyenage.fr/parisiens/metier.php?metier=184
http://parismoyenage.fr/parisiens/metier.php?met

In [27]:
len(persos)

28236

In [28]:
df = pd.DataFrame(persos)

In [29]:
df

,prenom,familyName,nom,startDate,endDate,positions,urlFiche
0,Prénom,Nom,Prénom Nom,de,à,Fonction,Plus d'infos
1,Nicolas,Arnoul,Nicolas Arnoul,1468,,changeur,Fiche
2,Jean,Arrode [4],Jean Arrode [4],1353,,changeur,Fiche
3,Nicolas,Arrode [7],Nicolas Arrode [7],1349,,changeur,Fiche
4,Jean,Auberteau,Jean Auberteau,1423,,changeur,Fiche
...,...,...,...,...,...,...,...
28231,Jean de,Meudon [5],Jean de Meudon [5],1333,,notaire et secrÃ©taire de Philippe d'Ãvreux r...,Fiche
28232,Pierre de,Meudon [1],Pierre de Meudon [1],1292,,concierge du SÃ©jour de Navarre,Fiche
28233,Vincent de,Montrosti [1],Vincent de Montrosti [1],,,secrÃ©taire de Pierre de Navarre,Fiche
28234,Jacquet de,Rue,Jacquet de Rue,1378,,chambellan du roi de Navarre,Fiche


In [32]:
persos = []

i = 0
j = 0

for index, row in df.iterrows():
    i = i +1
    if(i == 100):
        j = j + 1
        print(i*j)
        i = 0
    pp = row["nom"].replace("'"," ")
    pp = pp.replace(" [1]","")
    pp = pp.replace(" [2]","")
    pp = pp.replace(" [3]","")
    pp = pp.replace(" [4]","")
    
    try:
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""
        SELECT ?item ?itemLabel ?dateOfBirth ?dateOfDeath ?placeOfBirthLabel ?placeOfDeathLabel (GROUP_CONCAT(DISTINCT ?positionLabel;separator=", ") AS ?positions) 
        WHERE {
          SERVICE wikibase:mwapi {
              bd:serviceParam wikibase:api "EntitySearch" .
              bd:serviceParam wikibase:endpoint "www.wikidata.org" .
              bd:serviceParam mwapi:search '"""+pp+"""'.
              bd:serviceParam mwapi:language "fr" .
              ?item wikibase:apiOutputItem mwapi:item .
              ?num wikibase:apiOrdinal true .
          }
          ?item wdt:P31 wd:Q5.
          {?item wdt:P569 ?dateOfBirth.} UNION {MINUS {?item wdt:P569 ?dateOfBirth.}}
          {?item wdt:P570 ?dateOfDeath.} UNION {MINUS {?item wdt:P570 ?dateOfDeath.}}
          {?item wdt:P19 ?placeOfBirth.} UNION {MINUS {?item wdt:P19 ?placeOfBirth.}}
          {?item wdt:P20 ?placeOfDeath.} UNION {MINUS {?item wdt:P20 ?placeOfDeath.}}
          {?item wdt:P39 ?position.} UNION {MINUS {?item wdt:P39 ?position.}}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". 
                                  ?item rdfs:label ?itemLabel .
                                 ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                                 ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                                 ?position rdfs:label ?positionLabel .}

        } GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel
        LIMIT 1
        """)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        for r in results['results']['bindings']:
            perso = {}
            if("itemLabel" in r):
                perso['nom'] = r['itemLabel']['value']
            if("dateOfBirth" in r):
                perso['dateOfBirth'] = r['dateOfBirth']['value']
            if("placeOfBirthLabel" in r):
                perso['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
            if("dateOfDeath" in r):
                perso['dateOfDeath'] = r['dateOfDeath']['value']
            if("placeOfDeathLabel" in r):
                perso['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
            if("positions" in r):
                perso["positions"] = r["positions"]["value"]
            persos.append(perso)
    except HTTPError as err:
        i = i-1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [33]:
df = pd.DataFrame(persos)

In [35]:
#df.to_csv("../csv/parismoyenage/extract-persos-paris.csv", index=False)

In [63]:
df = pd.read_csv("../csv/parismoyenage/extract-persos-paris.csv")

In [94]:
df.shape

(3750, 8)

In [95]:
pd.options.display.max_rows = 5000
pd.options.display.max_colwidth = 5000

In [96]:
df.dtypes

nom                   object
dateOfBirth           object
placeOfBirthLabel     object
dateOfDeath           object
placeOfDeathLabel     object
positions             object
birthYear            float64
deathYear            float64
dtype: object

In [56]:
from datetime import datetime

In [76]:
df['birthYear'] = df['dateOfBirth'].map(lambda x: str(x).split("-")[0])


In [78]:
df['deathYear'] = df['dateOfDeath'].map(lambda x: str(x).split("-")[0])


In [79]:
toto = pd.to_numeric(df["birthYear"], errors="coerce")
tutu = pd.to_numeric(df["deathYear"], errors="coerce")
df["birthYear"] = toto
df["deathYear"] = tutu

In [105]:
df_p = df[(df["birthYear"] < 1500) | (df["deathYear"] < 1570)]

In [106]:
df_p

,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,positions,birthYear,deathYear
6,Simon de Dammartin,1180-01-01T00:00:00Z,NaN,1239-09-28T00:00:00Z,NaN,NaN,1180.0,1239.0
19,Guillaume Sanguin,NaN,NaN,1442-02-23T00:00:00Z,NaN,NaN,NaN,1442.0
20,Guillaume Sanguin,NaN,NaN,1442-02-23T00:00:00Z,NaN,NaN,NaN,1442.0
22,Nicolas Dipre,1460-01-01T00:00:00Z,Paris,1532-01-01T00:00:00Z,Paris,NaN,1460.0,1532.0
27,Guillaume Cousinot le Chancelier,1350-01-01T00:00:00Z,NaN,NaN,NaN,NaN,1350.0,NaN
28,Jean Cousin,1401-01-01T00:00:00Z,NaN,1599-01-01T00:00:00Z,NaN,NaN,1401.0,1599.0
29,Jean de Dammartin,1350-01-01T00:00:00Z,Jargeau,1454-01-01T00:00:00Z,Tours,NaN,1350.0,1454.0
30,Jean de Flandre,1250-01-01T00:00:00Z,NaN,1292-10-14T00:00:00Z,Namur,"évêque de Metz, prince-évêque de Liège",1250.0,1292.0
31,Pierre du Val,1500-01-01T00:00:00Z,NaN,1558-01-01T00:00:00Z,NaN,NaN,1500.0,1558.0
34,Jean Le Jeune,1411-01-01T00:00:00Z,Arras,1451-09-18T00:00:00Z,Rome,"cardinal, évêque catholique, évêque diocésain",1411.0,1451.0


In [107]:
df_p = df_p.drop_duplicates()

In [108]:
df_p

,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,positions,birthYear,deathYear
6,Simon de Dammartin,1180-01-01T00:00:00Z,NaN,1239-09-28T00:00:00Z,NaN,NaN,1180.0,1239.0
19,Guillaume Sanguin,NaN,NaN,1442-02-23T00:00:00Z,NaN,NaN,NaN,1442.0
22,Nicolas Dipre,1460-01-01T00:00:00Z,Paris,1532-01-01T00:00:00Z,Paris,NaN,1460.0,1532.0
27,Guillaume Cousinot le Chancelier,1350-01-01T00:00:00Z,NaN,NaN,NaN,NaN,1350.0,NaN
28,Jean Cousin,1401-01-01T00:00:00Z,NaN,1599-01-01T00:00:00Z,NaN,NaN,1401.0,1599.0
29,Jean de Dammartin,1350-01-01T00:00:00Z,Jargeau,1454-01-01T00:00:00Z,Tours,NaN,1350.0,1454.0
30,Jean de Flandre,1250-01-01T00:00:00Z,NaN,1292-10-14T00:00:00Z,Namur,"évêque de Metz, prince-évêque de Liège",1250.0,1292.0
31,Pierre du Val,1500-01-01T00:00:00Z,NaN,1558-01-01T00:00:00Z,NaN,NaN,1500.0,1558.0
34,Jean Le Jeune,1411-01-01T00:00:00Z,Arras,1451-09-18T00:00:00Z,Rome,"cardinal, évêque catholique, évêque diocésain",1411.0,1451.0
44,Roger Ier de Blois,NaN,NaN,1022-06-24T00:00:00Z,NaN,"évêque de Beauvais, Noyon et Senlis",NaN,1022.0


In [110]:
df_p.shape

(494, 8)

In [116]:
df_p[df_p["deathYear"].isnull()].count()

nom                  27
dateOfBirth          27
placeOfBirthLabel     9
dateOfDeath           2
placeOfDeathLabel     3
positions             6
birthYear            27
deathYear             0
dtype: int64

In [118]:
df_p.to_csv("../csv/parismoyenage/extract-persos-paris-clean.csv", index=False)